# RAG Fusion

Re-implemented from [this GitHub repo](https://github.com/Raudaschl/rag-fusion), all credit to the original author.

## Setup

For this example, we will use Pinecone and some fake data.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
index_name = os.getenv("PINECONE_INDEX_TWO")

#### Install Libraries

In [ ]:
# %pip install langchain_openai langchain_pinecone langchain_core langchain

#### Load Libraries

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

#### Load Fake Data

Here we define a set of fake documents related to climate change.

In [ ]:
all_documents = {
    "doc1": "Climate change and economic impact.",
    "doc2": "Public health concerns due to climate change.",
    "doc3": "Climate change: A social perspective.",
    "doc4": "Technological solutions to climate change.",
    "doc5": "Policy changes needed to combat climate change.",
    "doc6": "Climate change and its impact on biodiversity.",
    "doc7": "Climate change: The science and models.",
    "doc8": "Global warming: A subset of climate change.",
    "doc9": "How climate change affects daily weather.",
    "doc10": "The history of climate change activism.",
}

#### Create Vector Store

We will use Pinecone to create a vector store from these documents.



In [ ]:
vectorstore = PineconeVectorStore.from_texts(
    list(all_documents.values()), OpenAIEmbeddings(), index_name=index_name
)

## Define the Query Generator

We will now define a chain to do the query generation.

#### Load Libraries

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [ ]:
# %pip install langchainhub

#### Load Prompt From LangChainhub

In [ ]:
from langchain import hub

prompt = hub.pull("langchain-ai/rag-fusion-query-generation")

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),
#     ("user", "Generate multiple search queries related to: {original_query}"),
#     ("user", "OUTPUT (4 queries):")
# ])

#### Define Query Generation Chain

In [ ]:
generate_queries = (
    prompt | ChatOpenAI(temperature=0) | StrOutputParser() | (lambda x: x.split("\n"))
)

## Define the Full Chain

We can now put it all together and define the full chain. This chain:

1. Generates a bunch of queries
2. Looks up each query in the retriever
3. Joins all the results together using reciprocal rank fusion

Note that it does NOT do a final generation step.

#### Original Query

In [ ]:
original_query = "impact of climate change"

#### Create Retriever

In [ ]:
vectorstore = PineconeVectorStore.from_existing_index(index_name, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

#### Define Reciprocal Rank Fusion Function

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

#### Define Full Chain

In [ ]:
chain = generate_queries | retriever.map() | reciprocal_rank_fusion

#### Invoke Chain

In [ ]:
chain.invoke({"original_query": original_query})